<a href="https://colab.research.google.com/github/atikhasan007/Scikit-learn/blob/main/FINAL_6_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
print("Files added")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, UpSampling2D, concatenate, Input, Lambda, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import cv2
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [ ]:
def build_unet_model(input_size=(256, 256, 6)):
    inputs = Input(input_size)

    # Split the input into two separate RGB images
    input_image1 = inputs[..., :3]
    input_image2 = inputs[..., 3:]

    # Encoder for image 1
    c1_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_image1)
    c1_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1_1)
    c1_1 = Dropout(0.3)(c1_1)  # Dropout after convolutional blocks
    p1_1 = MaxPooling2D((2, 2))(c1_1)

    c2_1 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1_1)
    c2_1 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2_1)
    c2_1 = Dropout(0.3)(c2_1)  # Dropout after convolutional blocks
    p2_1 = MaxPooling2D((2, 2))(c2_1)

    c3_1 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2_1)
    c3_1 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3_1)
    c3_1 = Dropout(0.3)(c3_1)  # Dropout after convolutional blocks
    p3_1 = MaxPooling2D((2, 2))(c3_1)

    # Encoder for image 2 (same as image 1)
    c1_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(input_image2)
    c1_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1_2)
    c1_2 = Dropout(0.3)(c1_2)  # Dropout after convolutional blocks
    p1_2 = MaxPooling2D((2, 2))(c1_2)

    c2_2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1_2)
    c2_2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2_2)
    c2_2 = Dropout(0.3)(c2_2)  # Dropout after convolutional blocks
    p2_2 = MaxPooling2D((2, 2))(c2_2)

    c3_2 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2_2)
    c3_2 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3_2)
    c3_2 = Dropout(0.3)(c3_2)  # Dropout after convolutional blocks
    p3_2 = MaxPooling2D((2, 2))(c3_2)

    # Bottleneck (concatenate the features from both images)
    c4_1 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3_1)
    c4_1 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4_1)
    c4_1 = Dropout(0.4)(c4_1)  # Dropout after convolutional blocks

    c4_2 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3_2)
    c4_2 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4_2)
    c4_2 = Dropout(0.4)(c4_2)  # Dropout after convolutional blocks

    # Combine the features of both images at the bottleneck
    c4 = concatenate([c4_1, c4_2])

    # Decoder
    u5 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = concatenate([u5, c3_1, c3_2])  # Skip connection from both images encoders
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(c5)
    c5 = Dropout(0.3)(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c2_1, c2_2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = Dropout(0.3)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(c6)
    c6 = Dropout(0.3)(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c1_1, c1_2])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = Dropout(0.3)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(c7)
    c7 = Dropout(0.3)(c7)

    # Final output
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)
    model = Model(inputs, outputs)

    return model


In [ ]:
def build_unet_with_resnet(input_shape=(256, 256, 6)):
    # Split the input into two separate images (image1 and image2)
    inputs = Input(input_shape)
    input_image1 = inputs[..., :3]  # First 3 channels
    input_image2 = inputs[..., 3:]  # Last 3 channels

    # Pretrained ResNet50 as the backbone
    base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(256, 256, 3))

    # Extract encoder layers
    encoder_layers1 = [
        base_model.get_layer("conv1_relu").output,
        base_model.get_layer("conv2_block3_out").output,
        base_model.get_layer("conv3_block4_out").output,
        base_model.get_layer("conv4_block6_out").output,
        base_model.get_layer("conv5_block3_out").output
    ]

    # Create the encoder model for image1
    encoder1 = Model(inputs=base_model.input, outputs=encoder_layers1)
    encoder1.trainable = False  # Freeze pretrained layers initially

    # Extract features for image1
    encoder_outputs1 = encoder1(input_image1)

    # Extract features for image2 using the same encoder
    encoder_outputs2 = encoder1(input_image2)

    # Combine encoder outputs
    combined_features = [
        Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]), name=f"abs_diff_{i}")([feat1, feat2])
        for i, (feat1, feat2) in enumerate(zip(encoder_outputs1, encoder_outputs2))
    ]

    # Decoder with skip connections
    c1, c2, c3, c4, c5 = combined_features

    # Decoder 1
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Dropout(0.3)(c6)  # Add Dropout here
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = Dropout(0.3)(c6)  # Add Dropout here

    # Decoder 2
    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Dropout(0.3)(c7)  # Add Dropout here
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = Dropout(0.3)(c7)  # Add Dropout here

    # Decoder 3
    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Dropout(0.3)(c8)  # Add Dropout here
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = Dropout(0.3)(c8)  # Add Dropout here

    # Decoder 4
    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Dropout(0.3)(c9)  # Add Dropout here
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = Dropout(0.3)(c9)  # Add Dropout here

    # Final output
    u10 = UpSampling2D((2, 2))(c9)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(u10)

    model = Model(inputs, outputs)
    return model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# ===============================
# Build SiHDNet-like UNet model
def build_sihdnet_model(input_size=(256, 256, 6)):
    inputs = Input(input_size)

    # Split the input into two separate RGB images
    input_image1 = inputs[..., :3]
    input_image2 = inputs[..., 3:]

    # ---------------- Encoder for image 1 ----------------
    def encoder_block(x, filters):
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(x)
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(c)
        c = Dropout(0.3)(c)
        p = MaxPooling2D((2,2))(c)
        return c, p

    c1_1, p1_1 = encoder_block(input_image1, 64)
    c2_1, p2_1 = encoder_block(p1_1, 128)
    c3_1, p3_1 = encoder_block(p2_1, 256)

    # ---------------- Encoder for image 2 ----------------
    c1_2, p1_2 = encoder_block(input_image2, 64)
    c2_2, p2_2 = encoder_block(p1_2, 128)
    c3_2, p3_2 = encoder_block(p2_2, 256)

    # ---------------- Bottleneck ----------------
    c4_1 = Conv2D(512, (3,3), activation='relu', padding='same')(p3_1)
    c4_1 = Conv2D(512, (3,3), activation='relu', padding='same')(c4_1)
    c4_1 = Dropout(0.4)(c4_1)

    c4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(p3_2)
    c4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(c4_2)
    c4_2 = Dropout(0.4)(c4_2)

    # Combine features from both images
    c4 = concatenate([c4_1, c4_2])

    # ---------------- Decoder ----------------
    def decoder_block(x, skip1, skip2, filters):
        u = Conv2DTranspose(filters, (2,2), strides=(2,2), padding='same')(x)
        u = concatenate([u, skip1, skip2])
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(u)
        c = Dropout(0.3)(c)
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(c)
        c = Dropout(0.3)(c)
        return c

    c5 = decoder_block(c4, c3_1, c3_2, 256)
    c6 = decoder_block(c5, c2_1, c2_2, 128)
    c7 = decoder_block(c6, c1_1, c1_2, 64)

    # ---------------- Output ----------------
    outputs = Conv2D(1, (1,1), activation='sigmoid')(c7)
    model = Model(inputs, outputs)

    return model

# ===============================
# Instantiate SiHDNet model
model_sihdnet = build_sihdnet_model(input_size=(256, 256, 6))


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# ===============================
# Build GLAI-Net-like UNet model
def build_glainet_model(input_size=(256, 256, 6)):
    inputs = Input(input_size)

    # Split the input into two separate RGB images
    input_image1 = inputs[..., :3]
    input_image2 = inputs[..., 3:]

    # ---------------- Encoder for image 1 ----------------
    c1_1 = Conv2D(64, (3,3), activation='relu', padding='same')(input_image1)
    c1_1 = Conv2D(64, (3,3), activation='relu', padding='same')(c1_1)
    c1_1 = Dropout(0.3)(c1_1)
    p1_1 = MaxPooling2D((2,2))(c1_1)

    c2_1 = Conv2D(128, (3,3), activation='relu', padding='same')(p1_1)
    c2_1 = Conv2D(128, (3,3), activation='relu', padding='same')(c2_1)
    c2_1 = Dropout(0.3)(c2_1)
    p2_1 = MaxPooling2D((2,2))(c2_1)

    c3_1 = Conv2D(256, (3,3), activation='relu', padding='same')(p2_1)
    c3_1 = Conv2D(256, (3,3), activation='relu', padding='same')(c3_1)
    c3_1 = Dropout(0.3)(c3_1)
    p3_1 = MaxPooling2D((2,2))(c3_1)

    # ---------------- Encoder for image 2 ----------------
    c1_2 = Conv2D(64, (3,3), activation='relu', padding='same')(input_image2)
    c1_2 = Conv2D(64, (3,3), activation='relu', padding='same')(c1_2)
    c1_2 = Dropout(0.3)(c1_2)
    p1_2 = MaxPooling2D((2,2))(c1_2)

    c2_2 = Conv2D(128, (3,3), activation='relu', padding='same')(p1_2)
    c2_2 = Conv2D(128, (3,3), activation='relu', padding='same')(c2_2)
    c2_2 = Dropout(0.3)(c2_2)
    p2_2 = MaxPooling2D((2,2))(c2_2)

    c3_2 = Conv2D(256, (3,3), activation='relu', padding='same')(p2_2)
    c3_2 = Conv2D(256, (3,3), activation='relu', padding='same')(c3_2)
    c3_2 = Dropout(0.3)(c3_2)
    p3_2 = MaxPooling2D((2,2))(c3_2)

    # ---------------- Bottleneck ----------------
    c4_1 = Conv2D(512, (3,3), activation='relu', padding='same')(p3_1)
    c4_1 = Conv2D(512, (3,3), activation='relu', padding='same')(c4_1)
    c4_1 = Dropout(0.4)(c4_1)

    c4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(p3_2)
    c4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(c4_2)
    c4_2 = Dropout(0.4)(c4_2)

    c4 = concatenate([c4_1, c4_2])

    # ---------------- Decoder ----------------
    u5 = Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(c4)
    u5 = concatenate([u5, c3_1, c3_2])
    c5 = Conv2D(256, (3,3), activation='relu', padding='same')(u5)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3,3), activation='relu', padding='same')(c5)
    c5 = Dropout(0.3)(c5)

    u6 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
    u6 = concatenate([u6, c2_1, c2_2])
    c6 = Conv2D(128, (3,3), activation='relu', padding='same')(u6)
    c6 = Dropout(0.3)(c6)
    c6 = Conv2D(128, (3,3), activation='relu', padding='same')(c6)
    c6 = Dropout(0.3)(c6)

    u7 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
    u7 = concatenate([u7, c1_1, c1_2])
    c7 = Conv2D(64, (3,3), activation='relu', padding='same')(u7)
    c7 = Dropout(0.3)(c7)
    c7 = Conv2D(64, (3,3), activation='relu', padding='same')(c7)
    c7 = Dropout(0.3)(c7)

    # ---------------- Output ----------------
    outputs = Conv2D(1, (1,1), activation='sigmoid')(c7)
    model = Model(inputs, outputs)

    return model


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, Multiply, Add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# ===============================
# Build MTFSR-like UNet model with simple Dynamic Attention Unit
def build_mtfsr_model(input_size=(256, 256, 6)):
    inputs = Input(input_size)

    # Split the input into two separate RGB images
    input_image1 = inputs[..., :3]
    input_image2 = inputs[..., 3:]

    # ---------------- Encoder for image 1 ----------------
    def encoder_block(x, filters):
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(x)
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(c)
        c = Dropout(0.3)(c)
        p = MaxPooling2D((2,2))(c)
        return c, p

    c1_1, p1_1 = encoder_block(input_image1, 64)
    c2_1, p2_1 = encoder_block(p1_1, 128)
    c3_1, p3_1 = encoder_block(p2_1, 256)

    # ---------------- Encoder for image 2 ----------------
    c1_2, p1_2 = encoder_block(input_image2, 64)
    c2_2, p2_2 = encoder_block(p1_2, 128)
    c3_2, p3_2 = encoder_block(p2_2, 256)

    # ---------------- Bottleneck ----------------
    c4_1 = Conv2D(512, (3,3), activation='relu', padding='same')(p3_1)
    c4_1 = Conv2D(512, (3,3), activation='relu', padding='same')(c4_1)
    c4_1 = Dropout(0.4)(c4_1)

    c4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(p3_2)
    c4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(c4_2)
    c4_2 = Dropout(0.4)(c4_2)

    # ---------------- Dynamic Attention Unit ----------------
    attention = Multiply()([c4_1, c4_2])  # Simple feature interaction
    c4 = Add()([c4_1, c4_2, attention])  # Combine with residual

    # ---------------- Decoder ----------------
    def decoder_block(x, skip1, skip2, filters):
        u = Conv2DTranspose(filters, (2,2), strides=(2,2), padding='same')(x)
        u = concatenate([u, skip1, skip2])
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(u)
        c = Dropout(0.3)(c)
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(c)
        c = Dropout(0.3)(c)
        return c

    c5 = decoder_block(c4, c3_1, c3_2, 256)
    c6 = decoder_block(c5, c2_1, c2_2, 128)
    c7 = decoder_block(c6, c1_1, c1_2, 64)

    # ---------------- Output ----------------
    outputs = Conv2D(1, (1,1), activation='sigmoid')(c7)
    model = Model(inputs, outputs)

    return model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# ===============================
# Build SiHDNet-like UNet model
def build_sifanet_model(input_size=(256, 256, 6)):
    inputs = Input(input_size)

    # Split the input into two separate RGB images
    input_image1 = inputs[..., :3]
    input_image2 = inputs[..., 3:]

    # ---------------- Encoder for image 1 ----------------
    def encoder_block(x, filters):
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(x)
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(c)
        c = Dropout(0.3)(c)
        p = MaxPooling2D((2,2))(c)
        return c, p

    c1_1, p1_1 = encoder_block(input_image1, 64)
    c2_1, p2_1 = encoder_block(p1_1, 128)
    c3_1, p3_1 = encoder_block(p2_1, 256)

    # ---------------- Encoder for image 2 ----------------
    c1_2, p1_2 = encoder_block(input_image2, 64)
    c2_2, p2_2 = encoder_block(p1_2, 128)
    c3_2, p3_2 = encoder_block(p2_2, 256)

    # ---------------- Bottleneck ----------------
    c4_1 = Conv2D(512, (3,3), activation='relu', padding='same')(p3_1)
    c4_1 = Conv2D(512, (3,3), activation='relu', padding='same')(c4_1)
    c4_1 = Dropout(0.4)(c4_1)

    c4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(p3_2)
    c4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(c4_2)
    c4_2 = Dropout(0.4)(c4_2)

    # Combine features from both images
    c4 = concatenate([c4_1, c4_2])

    # ---------------- Decoder ----------------
    def decoder_block(x, skip1, skip2, filters):
        u = Conv2DTranspose(filters, (2,2), strides=(2,2), padding='same')(x)
        u = concatenate([u, skip1, skip2])
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(u)
        c = Dropout(0.3)(c)
        c = Conv2D(filters, (3,3), activation='relu', padding='same')(c)
        c = Dropout(0.3)(c)
        return c

    c5 = decoder_block(c4, c3_1, c3_2, 256)
    c6 = decoder_block(c5, c2_1, c2_2, 128)
    c7 = decoder_block(c6, c1_1, c1_2, 64)

    # ---------------- Output ----------------
    outputs = Conv2D(1, (1,1), activation='sigmoid')(c7)
    model = Model(inputs, outputs)

    return model

In [ ]:
input_shape = (256, 256, 6)

In [ ]:
model_unet = build_unet_model(input_shape)
model_unet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy', metrics=['accuracy'])

model_unet.summary()

In [ ]:
model_unet_with_resnet = build_unet_with_resnet(input_shape)
model_unet_with_resnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy', metrics=['accuracy'])

model_unet_with_resnet.summary()

In [ ]:
# ===============================
# Instantiate SiHDNet model
model_sihdnet = build_sihdnet_model(input_size=(256, 256, 6))

# Compile the model
model_sihdnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Show model summary
model_sihdnet.summary()

In [ ]:
# ===============================
# Instantiate GLAI-Net model
model_glainet = build_glainet_model(input_size=(256, 256, 6))

# Compile the model
model_glainet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Show model summary
model_glainet.summary()

In [ ]:
# ===============================
# Instantiate MTFSR model
model_mtfsr = build_mtfsr_model(input_size=(256, 256, 6))

# Compile the model
model_mtfsr.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Show model summary
model_mtfsr.summary()

In [ ]:
# ===============================
# Instantiate SifaNet model
model_sifanet = build_sifanet_model(input_size=(256, 256, 6))

# Compile the model
model_sifanet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Show model summary
model_sifanet.summary()

In [ ]:


# Paths to LEVIR-CD+ dataset
image1_train_dir = "/kaggle/input/levir-cd-change-detection/LEVIR-CD+/train/A"
image2_train_dir =  "/kaggle/input/levir-cd-change-detection/LEVIR-CD+/train/B"
mask_train_dir = "/kaggle/input/levir-cd-change-detection/LEVIR-CD+/train/label"

image1_test_dir =  "/kaggle/input/levir-cd-change-detection/LEVIR-CD+/train/A"
image2_test_dir =  "/kaggle/input/levir-cd-change-detection/LEVIR-CD+/train/B"
mask_test_dir = "/kaggle/input/levir-cd-change-detection/LEVIR-CD+/train/label"

input_shape = (256, 256)  # Resize dimensions

def load_images(image1_dir, image2_dir, mask_dir):
    image1_files = sorted(os.listdir(image1_dir))
    image2_files = sorted(os.listdir(image2_dir))
    mask_files = sorted(os.listdir(mask_dir))

    X = []
    y = []

    for img1, img2, mask in zip(image1_files, image2_files, mask_files):

        img1_path = os.path.join(image1_dir, img1)
        img2_path = os.path.join(image2_dir, img2)
        mask_path = os.path.join(mask_dir, mask)

        img1 = cv2.imread(img1_path)
        img2 = cv2.imread(img2_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Resize images and mask
        img1 = cv2.resize(img1, input_shape)
        img2 = cv2.resize(img2, input_shape)
        mask = cv2.resize(mask, input_shape)

        # Normalize images and mask
        img1 = img1 / 255.0
        img2 = img2 / 255.0
        mask = mask / 255.0

        # Stack images along the channel axis
        stacked_image = np.concatenate([img1, img2], axis=-1)  # Shape: (256, 256, 6)

        X.append(stacked_image)
        y.append(mask)

    return np.array(X), np.array(y)

# Load the dataset
X, y = load_images(image1_train_dir, image2_train_dir, mask_train_dir)
X_test, y_test = load_images(image1_test_dir, image2_test_dir, mask_test_dir)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Validation set size:", X_val.shape)
print("Test set size:", X_test.shape)



In [ ]:


callbacks_unet = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_change_detection_unet_model.keras", monitor="val_loss", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]



In [ ]:
callbacks_unet_with_resnet = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_change_detection_unet_resnet_model.keras", monitor="val_loss", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]


In [ ]:
# ===============================
# Callbacks for training
callbacks_sihdnet = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_change_detection_sihdnet_model.keras", monitor="val_loss", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

In [ ]:

# ===============================
# Callbacks for training
callbacks_glainet = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_change_detection_glainet_model.keras", monitor="val_loss", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]


In [ ]:
# ===============================
# Callbacks for training
callbacks_mtfsr = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_change_detection_mtfsr_model.keras", monitor="val_loss", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

In [ ]:
# ===============================
# Callbacks for training
callbacks_sifanet = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_change_detection_sihdnet_model.keras", monitor="val_loss", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

In [ ]:
history_unet = model_unet.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=25,
    verbose=1,
    callbacks=callbacks_unet
)



In [ ]:

history_unet_with_resnet = model_unet_with_resnet.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=25,
    verbose=1,
    callbacks=callbacks_unet_with_resnet
)


In [ ]:
# ===============================
# Training the model
history_sihdnet = model_sihdnet.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=25,
    verbose=1,
    callbacks=callbacks_sihdnet
)


In [ ]:
# ===============================
# Training the model
history_glainet = model_glainet.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=25,
    verbose=1,
    callbacks=callbacks_glainet
)

In [ ]:
# ===============================
# Training the model
history_mtfsr = model_mtfsr.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=25,
    verbose=1,
    callbacks=callbacks_mtfsr
)

In [ ]:
# ===============================
# Training the model
history_sifanet = model_sifanet.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=25,
    verbose=1,
    callbacks=callbacks_sifanet
)


In [ ]:
models = ['U-Net model', 'U-Net with RES-Net model','sihdnet model','glainet model','mtfsr model','sihdnet model']
history = [history_unet, history_unet_with_resnet ,history_sihdnet, history_glainet, history_mtfsr,history_sihdnet]
for i in range(6):
    for metric in ['loss', 'accuracy']:
        plt.plot(history[i].history[metric], label='Training Loss')
        plt.plot(history[i].history['val_' + metric], label='Validation Loss')
        plt.title(models[i] + ' ' + metric)
        plt.xlabel('Epochs')
        plt.ylabel(metric)
        plt.legend()
        plt.show()

In [ ]:


# Evaluate the model
test_loss, test_accuracy = model_unet_with_resnet.evaluate(X_test, y_test, batch_size=8, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test set
predictions_unet_with_resnet = model_unet_with_resnet.predict(X_test, batch_size=8, verbose=1)
#predicted_masks = (predictions > 0.5).astype(int)



In [ ]:
# Evaluate the model
test_loss, test_accuracy = model_unet.evaluate(X_test, y_test, batch_size=8, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test set
predictions_unet = model_unet.predict(X_test, batch_size=8, verbose=1)
#predicted_masks = (predictions > 0.5).astype(int)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model_sihdnet.evaluate(X_test, y_test, batch_size=8, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test set
predictions_unet_sihdnet = model_sihdnet.predict(X_test, batch_size=8, verbose=1)
#predicted_masks = (predictions > 0.5).astype(int)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model_glainet.evaluate(X_test, y_test, batch_size=8, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test set
predictions_unet_glainet = model_glainet.predict(X_test, batch_size=8, verbose=1)
#predicted_masks = (predictions > 0.5).astype(int)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model_mtfsr.evaluate(X_test, y_test, batch_size=8, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test set
predictions_unet_mtfsr = model_mtfsr.predict(X_test, batch_size=8, verbose=1)
#predicted_masks = (predictions > 0.5).astype(int)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model_sifanet.evaluate(X_test, y_test, batch_size=8, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on test set
predictions_unet_sifanet = model_sifanet.predict(X_test, batch_size=8, verbose=1)
#predicted_masks = (predictions > 0.5).astype(int)

In [ ]:


def overlay_mask(image, mask, color=(255, 0, 0), alpha=0.5, max_val = 0.2):

    mask_colored = np.zeros_like(image, dtype=np.uint8)
    mask_colored[mask > max_val] = color

    overlayed = cv2.addWeighted(image, 1, mask_colored, alpha, 0)

    return overlayed


predictions = [predictions_unet_with_resnet, predictions_unet,predictions_unet_sihdnet,predictions_unet_glainet,predictions_unet_mtfsr,predictions_unet_sifanet]
models = ['U-Net model', 'U-Net with RES-Net model','sihdnet model','glainet model','mtfsr model','sihdnet model']

for k in range(6):
    print('-----------------------------'+ models[k] +'----------------------------------------')

    # Visualize results
    for i in range(6):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 4, 1)
        plt.imshow(X_test[i, :, :, :3])
        plt.title("Input Image 1")
        plt.subplot(1, 4, 2)
        plt.imshow(X_test[i, :, :, 3:])
        plt.title("Input Image 2")
        plt.subplot(1, 4, 3)
        plt.imshow(y_test[i].squeeze(), cmap='gray')
        plt.title("Ground Truth Mask")
        plt.subplot(1, 4, 4)
        plt.imshow(predictions[k][i].squeeze(), cmap='gray')
        plt.title("Predicted Mask")
        plt.show()
        print(f"Predictions Min: {predictions[k][i].min()}, Max: {predictions[k][i].max()}")


        # Overlay mask on image
        overlayed_image_1 = overlay_mask((X_test[i, :, :, :3] * 255).astype(np.uint8), predictions[k][i].squeeze(), color=(255, 0, 0), alpha=0.5)
        overlayed_image_2 = overlay_mask((X_test[i, :, :, 3:] * 255).astype(np.uint8), predictions[k][i].squeeze(), color=(255, 0, 0), alpha=0.5)

        # Display results
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.imshow(overlayed_image_1)
        plt.title("Overlayed Image - 1")

        plt.subplot(1, 2, 2)
        plt.imshow(overlayed_image_2)
        plt.title("Overlayed Image - 2")

        plt.show()



In [ ]:
import os

# Path to save the best model
best_model_path = '/kaggle/working/sihdnet_best_model.h5'

# Check if model already exists
if os.path.exists(best_model_path):
    print("Best model is already saved")
else:
    model_sihdnet.save(best_model_path)
    print("Best model (SiHDNet) saved successfully")

In [ ]:
best_weights_path = '/kaggle/working/sihdnet_best_model_weights.weights.h5'
import os

# Path to save the best model weights
best_weights_path = '/kaggle/working/sihdnet_best_model_weights.weights.h5'

# Check if weights file already exists
if os.path.exists(best_weights_path):
    print("Best model weights are already saved")
else:
    # Save only weights of the best model (SiHDNet)
    model_sihdnet.save_weights(best_weights_path)
    print("Best model weights (SiHDNet) saved successfully")